# Experiments with Graph CNNs

## Setup

In [1]:
!pip install -U -q PyDrive
!pip install torchvision
!pip install --verbose --no-cache-dir torch-scatter
!pip install --verbose --no-cache-dir torch-sparse
!pip install --verbose --no-cache-dir torch-cluster
!pip install --verbose --no-cache-dir torch-spline-conv (optional)
!pip install torch-geometric

     |████████████████████████████████| 993kB 5.1MB/s 
Created temporary directory: /tmp/pip-ephem-wheel-cache-crrijc0x
Created temporary directory: /tmp/pip-req-tracker-c_4betcs
Created requirements tracker '/tmp/pip-req-tracker-c_4betcs'
Created temporary directory: /tmp/pip-install-nr_0s6d0
  1 location(s) to search for versions of torch-scatter:
  * https://pypi.org/simple/torch-scatter/
  Getting page https://pypi.org/simple/torch-scatter/
  Starting new HTTPS connection (1): pypi.org:443
  https://pypi.org:443 "GET /simple/torch-scatter/ HTTP/1.1" 200 929
  Analyzing links from page https://pypi.org/simple/torch-scatter/
    Found link https://files.pythonhosted.org/packages/29/96/566ac314e796d4b07209a3b88cc7a8d2e8582d55819e33f72e6c0e8d8216/torch_scatter-0.3.0.tar.gz#sha256=9e5e5a6efa4ef45f584e8611f83690d799370dd122b862646751ae112b685b50 (from https://pypi.org/simple/torch-scatter/), version: 0.3.0
    Found link https://files.pythonhosted.org/packages/6a/b0/ecffacddf573c147c70c6

In [0]:
# PyDrive (https://colab.research.google.com/notebooks/io.ipynb#scrollTo=7taylj9wpsA2)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
import pandas as pd



# %matplotlib inline

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Data handling on graphs

In [10]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
#List of some methods of torch_geometric.data.Data
print(data)
print(data.keys)
print(data['x'])
print(data.num_nodes)
print(data.num_edges)
print(data.num_features)
print(data.contains_isolated_nodes())
print(data.contains_self_loops())
print(data.is_directed())


Data(edge_index=[2, 4], x=[3, 1])
['x', 'edge_index']
tensor([[-1.],
        [ 0.],
        [ 1.]])
3
4
1
False
False
False


# Learning methods on graphs

In [11]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [0]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [15]:
print(dataset[0])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
print('Training finished!')

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
Training finished!


In [16]:
#Evaluate model
model.eval()
_, pred = model(data).max(dim=1)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7980


# Message Passing Networks

In [0]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

In [20]:
conv = GCNConv(16, 32)
x = conv(x, edge_index)

RuntimeError: ignored